# Git commands


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/My Drive/Colab Notebooks/Master_course_Homeworks/
# تنظیم ایمیل و نام (اگر سشن جدید است)
!git config --global user.email "arshia.mokhlesi@gmail.com"
!git config --global user.name "arshia.mkh"

# ساخت برنچ برای تمرین جدید
# !git checkout -b hw6-IR

/content/drive/My Drive/Colab Notebooks/Master_course_Homeworks


In [6]:
!git add .
!git commit -m "complete Part2:Conversational Search System: section[1-6]"
!git push origin hw6-IR

[hw6-IR 7427551] complete Part2:Conversational Search System: section[1-6]
 4 files changed, 27 insertions(+), 27 deletions(-)
 rewrite Intelligence Information Retrieval/HW6/Conversational_Search_System.ipynb (95%)
 rewrite Intelligence Information Retrieval/HW6/IIR_CA6.ipynb (95%)
 rewrite Intelligence Information Retrieval/HW6/rewritten_queries.csv (99%)
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 9.66 KiB | 299.00 KiB/s, done.
Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/ArsHia-cdMstr/Master_course_Homeworks.git
   b62c781..7427551  hw6-IR -> hw6-IR


In [7]:
!git checkout main
!git pull origin main
!git merge hw6-IR
!git push origin main

error: Your local changes to the following files would be overwritten by checkout:
	Intelligence Information Retrieval/HW6/Conversational_Search_System.ipynb
Please commit your changes or stash them before you switch branches.
Aborting
From https://github.com/ArsHia-cdMstr/Master_course_Homeworks
 * branch            main       -> FETCH_HEAD
Already up to date.
Already up to date.
Everything up-to-date
